In [4]:
import pandas as pd
from scipy.stats import ttest_ind, f_oneway
import plotly.express as px

Read documents

In [14]:
data = pd.read_csv("../data/2-Processed/Forms-Results-Translated.csv")
data.sample(3)


,ID,Age,Occupation,Gender,Education Level,Country,How often do you use AI tools?,What do you primarily use AI tools for?,Do you feel more productive when using AI tools?,Do you believe overuse of AI can negatively affect mental health or problem-solving skills?,Do you know someone that use AI as conversational partner?,I feel more confident in solving problems on my own without AI assistance.,Using AI makes me feel mentally lazy.,Relying on AI helps reduce my stress.,AI tools make me second-guess my own knowledge or decisions.,Its recomendable for people to use AI as conversational partner,What application with AI would you like to exist?
7,7,25-34,Business Development Analyst,Female,Master’s Degree,Brazil,Daily,['Studying or learning'],Yes,Yes,No,Disagree,Disagree,Neutral,Neutral,Neutral,NaN
24,24,18-24,Software developer,Female,Bachelor’s Degree,Argentina,Daily,['Work/professional tasks'],Yes,Yes,Yes,Strongly Agree,Agree,Neutral,Disagree,Strongly Disagree,Una aplicación que se encargará de la sección ...
43,43,25-34,Illustrator,Male,Bachelor’s Degree,Andorra,Rarely,"[""Don't use""]",No,Yes,No,Neutral,Agree,Neutral,Agree,Neutral,NaN


In [6]:
risk = pd.read_csv("../data/2-Processed/Computed_Risk_Values.csv")
risk.head()

,Self-Acceptance,Positive Relations with Others,Autonomy,Environmental Mastery,Personal Growth
0,3.333333,3.5,4.000000,4.333333,2.0
1,2.000000,2.0,2.000000,4.000000,2.0
2,4.333333,3.5,4.333333,4.000000,3.0
3,3.000000,3.0,3.333333,3.666667,2.0
4,3.000000,1.5,4.000000,1.333333,3.0


# Studies

- Different use of compute risk by gender

In [7]:
# Put the gender column from one to other

genderRelated = risk
genderRelated['Gender'] = data['Gender']

genderRelated.head()

,Self-Acceptance,Positive Relations with Others,Autonomy,Environmental Mastery,Personal Growth,Gender
0,3.333333,3.5,4.000000,4.333333,2.0,Female
1,2.000000,2.0,2.000000,4.000000,2.0,Male
2,4.333333,3.5,4.333333,4.000000,3.0,Female
3,3.000000,3.0,3.333333,3.666667,2.0,Female
4,3.000000,1.5,4.000000,1.333333,3.0,Trans


In [8]:
newRelated = genderRelated.groupby('Gender').mean(numeric_only=True)

fig = px.imshow(newRelated, 
                 labels=dict(x="Ryff Scale Questions", y="Gender", color="Risk"),
                text_auto=True,  zmin = 0, zmax= 4,height=500)
fig.update_xaxes(side="top")
fig.show()

- Different use of compute risk by Age

In [9]:
ageRelated = risk
ageRelated['Age'] = data['Age']

ageRelated.head()

,Self-Acceptance,Positive Relations with Others,Autonomy,Environmental Mastery,Personal Growth,Gender,Age
0,3.333333,3.5,4.000000,4.333333,2.0,Female,25-34
1,2.000000,2.0,2.000000,4.000000,2.0,Male,25-34
2,4.333333,3.5,4.333333,4.000000,3.0,Female,25-34
3,3.000000,3.0,3.333333,3.666667,2.0,Female,18-24
4,3.000000,1.5,4.000000,1.333333,3.0,Trans,18-24


In [10]:
newRelated = ageRelated.groupby('Age').mean(numeric_only=True)

fig = px.imshow(newRelated, 
                 labels=dict(x="Ryff Scale Questions", y="Age", color="Risk"),
                text_auto=True,  zmin = 0, zmax= 4,height=900)
fig.update_xaxes(side="top")
fig.show()

# Hypothesis

- Frequent users of AI tools are more likely to report beeing less stress.

Null Hypothesis (H₀): There is no difference in self-reported less stress between frequent and infrequent users.

Alternative Hypothesis (H₁): Frequent users report significantly higher feelings of less stress.

In [ ]:
likert_map = {
    'Strongly Disagree': 1,
    'Disagree': 2,
    'Neutral': 3,
    'Agree': 4,
    'Strongly Agree': 5
}

data['stress_reduction_score'] = data['Relying on AI helps reduce my stress.'].map(likert_map)
data['usage_group'] = data['How often do you use AI tools?'].map({
    'Daily': 'Frequent',
    'Several times a week': 'Frequent',
    'Occasionally': 'Infrequent',
    'Rarely': 'Infrequent',
    'Never': 'Infrequent'
})


df_clean = data.dropna(subset=['stress_reduction_score', 'usage_group'])

frequent = df_clean[df_clean['usage_group'] == 'Frequent']['stress_reduction_score']
infrequent = df_clean[df_clean['usage_group'] == 'Infrequent']['stress_reduction_score']

t_stat, p_value = ttest_ind(frequent, infrequent, equal_var=False)

p_one_tailed = p_value / 2 if t_stat > 0 else 1 - (p_value / 2)

confidence_level = (1 - p_one_tailed) * 100

print(f"T-statistic: {t_stat:.4f}")
print(f"One-tailed p-value: {p_one_tailed:.4f}")
print(f"Confidence Level: {confidence_level:.2f}%")

if p_one_tailed < 0.05:
    print("✅ Statistically significant: Frequent users have less stress.")
else:
    print("❌ Not statistically significant: No strong evidence that frequent users have less stress.")


T-statistic: 3.8639
One-tailed p-value: 0.0003
Confidence Level: 99.97%
✅ Statistically significant: Frequent users report more mental laziness.
